In [1]:
import cv2 as cv
import imutils
import sys
import numpy as np
import matplotlib.pyplot as plt
import time
from tabulate import tabulate

fr_proc_time = []
fr_scale_time = []
iter_time = []

In [2]:
def contrast_enh(img, p):
    chans = cv.split(img)
    for channel in chans:
        hist = cv.calcHist([channel], [0], None, [256], [0, 256])
        threshold = max(hist)*p
        
        for first in range(hist.size):
            if hist[first] > threshold:
                break;

        for last in range(255,-1,-1):
            if hist[last] > threshold:
                break;

        if last == first:
            den = 0.1
        else:
            den = last - first
            
        scale = 255 / den
        offset = first

        channel[channel < first] = 0
        channel[channel > last] = 255
        channel[(channel >= first) & (channel <= last)] = channel[(channel >= first) & (channel <= last)] * scale - offset
        
        final_image = cv.merge([chans[0], chans[1], chans[2]])
    
    return final_image

def image_sharp(image):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])

    # Apply the filter matrix to the image
    sharpened = cv.filter2D(image, -1, kernel)
    
    return sharpened

In [4]:
cap = cv.VideoCapture("IMAGES/parrot-video.mp4")
if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    loop_start = time.time()
    # Capture frame-by-frame
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    START = time.time()
    scale_percent = 30  # Puoi regolare questa percentuale in base alle tue esigenze
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    dim = (width, height)
    frame = cv.resize(frame, dim, interpolation=cv.INTER_AREA)
    STOP = time.time()
    fr_scale_time.append(STOP - START)  
    
    START = time.time()
    # Our operations on the frame come here
    image = contrast_enh(frame, 0.05)
    #image = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    STOP = time.time()
    fr_proc_time.append(STOP - START)  
    
    # Display the resulting frame
    cv.imshow('frame', image)
    if cv.waitKey(1) == ord('q'):
        break
    
    loop_end = time.time()
    iter_time.append(loop_end - loop_start)

# When everything done, release the capture
cap.release()
cv.destroyAllWindows()

Can't receive frame (stream end?). Exiting ...


In [5]:
table_data = []
for i, (time1, time2, itime) in enumerate(zip(fr_proc_time, fr_scale_time, iter_time)):
    table_data.append([i, time1, time2, itime])

# Stampa la tabella
headers = ["Frame", "Elaboration Time", "Scaling Time", "Loop Iteration Time"]
table_str = tabulate(table_data, headers, tablefmt="pretty")
print(table_str)

+-------+----------------------+----------------------+---------------------+
| Frame |   Elaboration Time   |     Scaling Time     | Loop Iteration Time |
+-------+----------------------+----------------------+---------------------+
|   0   | 0.06904125213623047  | 0.015633583068847656 | 0.4130253791809082  |
|   1   | 0.06987714767456055  | 0.014211893081665039 | 0.1539754867553711  |
|   2   | 0.07815098762512207  | 0.021670103073120117 | 0.1469557285308838  |
|   3   |  0.0534977912902832  | 0.031250715255737305 | 0.12255430221557617 |
|   4   | 0.06746673583984375  | 0.037178754806518555 | 0.16670942306518555 |
|   5   | 0.06881856918334961  | 0.030318498611450195 | 0.15205740928649902 |
|   6   | 0.06400036811828613  | 0.02845931053161621  | 0.15143632888793945 |
|   7   |  0.0794074535369873  | 0.03034663200378418  | 0.1549375057220459  |
|   8   | 0.06362581253051758  | 0.02843761444091797  | 0.1329481601715088  |
|   9   | 0.055440664291381836 | 0.02470564842224121  | 0.132993

In [6]:
frame_proc = np.array(fr_proc_time)
frame_scale = np.array(fr_scale_time)
frame_iter = np.array(iter_time)

avg_proc = frame_proc.mean()
avg_scale = frame_scale.mean()
var_proc = frame_proc.var()
var_scale = frame_scale.var()
avg_iter = frame_iter.mean()
var_iter = frame_iter.var()

table_data = [
    ["Frame Processing Time", avg_proc, var_proc],
    ["Frame Scaling Time", avg_scale, var_scale],
    ["Iteration Time", avg_iter, var_iter]
]

# Specifica gli header della tabella
headers = ["Metric", "Average [s]", "Variance [s]"]

# Stampa la tabella
table_str = tabulate(table_data, headers, tablefmt="pretty")

# Stampa il risultato
print(f'IMAGE SIZE: {width}x{height} pixels')
print(table_str)



IMAGE SIZE: 1152x648 pixels
+-----------------------+----------------------+------------------------+
|        Metric         |     Average [s]      |      Variance [s]      |
+-----------------------+----------------------+------------------------+
| Frame Processing Time | 0.06751426484849718  | 5.494956165958335e-05  |
|  Frame Scaling Time   | 0.030381506813897025 | 5.746469236149287e-05  |
|    Iteration Time     |  0.1509724966684977  | 0.00044480435196615477 |
+-----------------------+----------------------+------------------------+
